# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
import sys
!{sys.executable} -m pip install citipy

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,59.25,96,75,11.50,broken clouds
1,1,Porto Belo,BR,-27.1578,-48.5531,71.55,91,78,11.27,broken clouds
2,2,Kokopo,PG,-4.3520,152.2633,84.20,66,100,6.58,light rain
3,3,Kapaa,US,22.0752,-159.3190,82.38,78,20,13.80,few clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,58.30,97,100,2.37,overcast clouds


In [6]:
! pwd && ls -al



/Users/noemimontelongo/Desktop/World_Weather_Analysis/Vacation_Search
total 272
drwxr-xr-x   7 noemimontelongo  staff     224 Oct 26 00:19 .
drwxr-xr-x  19 noemimontelongo  staff     608 Oct 26 21:49 ..
-rw-r--r--@  1 noemimontelongo  staff    6148 Oct 25 10:48 .DS_Store
drwxr-xr-x   3 noemimontelongo  staff      96 Oct 20 14:44 .ipynb_checkpoints
-rw-r--r--@  1 noemimontelongo  staff  125704 Oct 26 00:19 Vacation_Search.ipynb
drwxr-xr-x   4 noemimontelongo  staff     128 Oct 20 17:24 __pycache__
-rw-r--r--@  1 noemimontelongo  staff      70 Oct 20 16:49 config.py


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [7]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kokopo,PG,-4.3520,152.2633,84.20,66,100,6.58,light rain
3,3,Kapaa,US,22.0752,-159.3190,82.38,78,20,13.80,few clouds
20,20,Boyolangu,ID,-8.1181,111.8935,81.30,71,79,10.71,broken clouds
35,35,Hithadhoo,MV,-0.6000,73.0833,82.31,75,29,7.20,scattered clouds
37,37,Mandera,KE,3.9366,41.8670,77.49,77,14,13.31,few clouds
44,44,Atuona,PF,-9.8000,-139.0333,77.20,77,14,19.06,few clouds
45,45,Buala,SB,-8.1450,159.5921,83.41,73,67,2.39,moderate rain
47,47,Kahului,US,20.8947,-156.4700,82.42,80,40,13.80,scattered clouds
48,48,Pilani,IN,28.3684,75.6041,77.47,32,0,7.67,clear sky
50,50,Vaini,TO,-21.2000,-175.2000,80.76,65,75,9.22,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [9]:
# Drop any empty rows in the preferred_cities_df DataFrame and 
preferred_cities_df.isnull()



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
20,False,False,False,False,False,False,False,False,False,False
35,False,False,False,False,False,False,False,False,False,False
37,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
671,False,False,False,False,False,False,False,False,False,False
672,False,False,False,False,False,False,False,False,False,False
677,False,False,False,False,False,False,False,False,False,False
680,False,False,False,False,False,False,False,False,False,False


In [10]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kokopo,PG,-4.3520,152.2633,84.20,66,100,6.58,light rain
3,3,Kapaa,US,22.0752,-159.3190,82.38,78,20,13.80,few clouds
20,20,Boyolangu,ID,-8.1181,111.8935,81.30,71,79,10.71,broken clouds
35,35,Hithadhoo,MV,-0.6000,73.0833,82.31,75,29,7.20,scattered clouds
37,37,Mandera,KE,3.9366,41.8670,77.49,77,14,13.31,few clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Keti Bandar,PK,24.1447,67.4497,79.54,76,0,5.59,clear sky
672,672,Inhambane,MZ,-23.8650,35.3833,76.98,88,40,3.44,scattered clouds
677,677,Grand Gaube,MU,-20.0064,57.6608,75.60,71,78,3.00,broken clouds
680,680,Ciudad Bolivar,VE,8.1222,-63.5497,79.36,82,95,5.12,overcast clouds


In [12]:
#create a new DataFrame.
clean_travel_cities = pd.DataFrame(preferred_cities_df.dropna())

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kokopo,PG,-4.3520,152.2633,84.20,66,100,6.58,light rain
3,3,Kapaa,US,22.0752,-159.3190,82.38,78,20,13.80,few clouds
20,20,Boyolangu,ID,-8.1181,111.8935,81.30,71,79,10.71,broken clouds
35,35,Hithadhoo,MV,-0.6000,73.0833,82.31,75,29,7.20,scattered clouds
37,37,Mandera,KE,3.9366,41.8670,77.49,77,14,13.31,few clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Keti Bandar,PK,24.1447,67.4497,79.54,76,0,5.59,clear sky
672,672,Inhambane,MZ,-23.8650,35.3833,76.98,88,40,3.44,scattered clouds
677,677,Grand Gaube,MU,-20.0064,57.6608,75.60,71,78,3.00,broken clouds
680,680,Ciudad Bolivar,VE,8.1222,-63.5497,79.36,82,95,5.12,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [13]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description
2,Kokopo,PG,84.20,-4.3520,152.2633,light rain
3,Kapaa,US,82.38,22.0752,-159.3190,few clouds
20,Boyolangu,ID,81.30,-8.1181,111.8935,broken clouds
35,Hithadhoo,MV,82.31,-0.6000,73.0833,scattered clouds
37,Mandera,KE,77.49,3.9366,41.8670,few clouds
...,...,...,...,...,...,...
671,Keti Bandar,PK,79.54,24.1447,67.4497,clear sky
672,Inhambane,MZ,76.98,-23.8650,35.3833,scattered clouds
677,Grand Gaube,MU,75.60,-20.0064,57.6608,broken clouds
680,Ciudad Bolivar,VE,79.36,8.1222,-63.5497,overcast clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [14]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df = preferred_cities_df.dropna()[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Kokopo,PG,84.20,-4.3520,152.2633,light rain,
3,Kapaa,US,82.38,22.0752,-159.3190,few clouds,
20,Boyolangu,ID,81.30,-8.1181,111.8935,broken clouds,
35,Hithadhoo,MV,82.31,-0.6000,73.0833,scattered clouds,
37,Mandera,KE,77.49,3.9366,41.8670,few clouds,
44,Atuona,PF,77.20,-9.8000,-139.0333,few clouds,
45,Buala,SB,83.41,-8.1450,159.5921,moderate rain,
47,Kahului,US,82.42,20.8947,-156.4700,scattered clouds,
48,Pilani,IN,77.47,28.3684,75.6041,clear sky,
50,Vaini,TO,80.76,-21.2000,-175.2000,broken clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [15]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [16]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Kokopo,PG,84.20,-4.3520,152.2633,light rain,Kokopo Beach Bungalow Resort
3,Kapaa,US,82.38,22.0752,-159.3190,few clouds,Sheraton Kauai Coconut Beach Resort
20,Boyolangu,ID,81.30,-8.1181,111.8935,broken clouds,Yos ega cv
35,Hithadhoo,MV,82.31,-0.6000,73.0833,scattered clouds,Scoop Guest House
37,Mandera,KE,77.49,3.9366,41.8670,few clouds,Red Sea Resort & Suites Mandera County


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [18]:
hotel_df.isnull()


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
20,False,False,False,False,False,False,False
35,False,False,False,False,False,False,False
37,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
671,False,False,False,False,False,False,False
672,False,False,False,False,False,False,False
677,False,False,False,False,False,False,False
680,False,False,False,False,False,False,False


In [19]:
# Drop the rows where there is no Hotel Name.
#clean_hotel_df = # YOUR CODE HERE
#df = df.drop(df[df.score < 50].index)

clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == True].index)


# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Kokopo,PG,84.20,-4.3520,152.2633,light rain,Kokopo Beach Bungalow Resort
3,Kapaa,US,82.38,22.0752,-159.3190,few clouds,Sheraton Kauai Coconut Beach Resort
20,Boyolangu,ID,81.30,-8.1181,111.8935,broken clouds,Yos ega cv
35,Hithadhoo,MV,82.31,-0.6000,73.0833,scattered clouds,Scoop Guest House
37,Mandera,KE,77.49,3.9366,41.8670,few clouds,Red Sea Resort & Suites Mandera County
...,...,...,...,...,...,...,...
671,Keti Bandar,PK,79.54,24.1447,67.4497,clear sky,Haji laloo memon house
672,Inhambane,MZ,76.98,-23.8650,35.3833,scattered clouds,Oceano
677,Grand Gaube,MU,75.60,-20.0064,57.6608,broken clouds,Veranda Paul et Virginie Hotel & Spa
680,Ciudad Bolivar,VE,79.36,8.1222,-63.5497,overcast clouds,Hotel Edi


In [20]:
# Create an CSV file to store the clean_hotel_df

**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [21]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [22]:
# Set the file name.
output_data_file = "../WeatherPY_vacation.csv"


# Create a CSV file by using the clean_hotel_df DataFrame
hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [23]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [24]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Kokopo,PG,84.20,-4.3520,152.2633,light rain,Kokopo Beach Bungalow Resort
3,Kapaa,US,82.38,22.0752,-159.3190,few clouds,Sheraton Kauai Coconut Beach Resort
20,Boyolangu,ID,81.30,-8.1181,111.8935,broken clouds,Yos ega cv
35,Hithadhoo,MV,82.31,-0.6000,73.0833,scattered clouds,Scoop Guest House
37,Mandera,KE,77.49,3.9366,41.8670,few clouds,Red Sea Resort & Suites Mandera County
...,...,...,...,...,...,...,...
671,Keti Bandar,PK,79.54,24.1447,67.4497,clear sky,Haji laloo memon house
672,Inhambane,MZ,76.98,-23.8650,35.3833,scattered clouds,Oceano
677,Grand Gaube,MU,75.60,-20.0064,57.6608,broken clouds,Veranda Paul et Virginie Hotel & Spa
680,Ciudad Bolivar,VE,79.36,8.1222,-63.5497,overcast clouds,Hotel Edi


In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Display sample data
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Kokopo Beach Bungalow Resort</dd>\n<dt>City</dt><dd>Kokopo</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Weather</dt><dd>light rain and 84.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.38 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Yos ega cv</dd>\n<dt>City</dt><dd>Boyolangu</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>broken clouds and 81.3 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Scoop Guest House</dd>\n<dt>City</dt><dd>Hithadhoo</dd>\n<dt>Country</dt><dd>MV</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 82.31 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Red Sea Resort & Suites Mandera County</dd>\n<dt>City</dt><dd>Mandera</dd>\n<dt>Country</dt><dd>KE</dd>\n<dt>Current Weather</dt><dd>few clouds and 77.49 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [27]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
2,-4.3520,152.2633
3,22.0752,-159.3190
20,-8.1181,111.8935
35,-0.6000,73.0833
37,3.9366,41.8670
44,-9.8000,-139.0333
45,-8.1450,159.5921
47,20.8947,-156.4700
48,28.3684,75.6041
50,-21.2000,-175.2000


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [28]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


# Create a figure to add the Google map as a layer
# YOUR CODE HERE
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
# YOUR CODE HERE
fig

Figure(layout=FigureLayout(height='420px'))